$$
\begin{align*}
x(t) &= [\sqrt{D^2 - z^2}]\cos(t) + x_1\\
y(t) &= [\sqrt{D^2 - z^2}]\sin(t) + y_1\\

r(t) &= \sqrt{[D^2 - z^2]\cos^2(t) + x_1^2 + x_1[\sqrt{D^2 - z^2}]\cos(t) + [D^2 - z^2]\sin^2(t) + y_1^2 + y_1[\sqrt{D^2 - z^2}]\sin(t) + z^2}\\
&= \sqrt{[D^2 - z^2] + x_1^2 + y_1^2 + z^2 + [\sqrt{D^2 - z^2}](x_1\cos(t)+y_1\sin(t))}\\
&= \sqrt{D^2 + x_1^2 + y_1^2 + [\sqrt{D^2 - z^2}](x_1\cos(t)+y_1\sin(t))}\\

\frac{dx}{dt} &= -[\sqrt{D^2 - z^2}]\sin(t)\\
\frac{dy}{dt} &= [\sqrt{D^2 - z^2}]\cos(t)\\

{\begin{bmatrix} \frac{dx}{dt} \end{bmatrix}}^2 &= (D^2 - z^2)\sin^2(t)\\
{\begin{bmatrix} \frac{dy}{dt} \end{bmatrix}}^2 &= (D^2 - z^2)\cos^2(t)\\

{\begin{bmatrix} \frac{dx}{dt} \end{bmatrix}}^2 + {\begin{bmatrix} \frac{dy}{dt} \end{bmatrix}}^2 &= (D^2 - z^2)\sin^2(t) + (D^2 - z^2)\cos^2(t)\\
{\begin{bmatrix} \frac{dx}{dt} \end{bmatrix}}^2 + {\begin{bmatrix} \frac{dy}{dt} \end{bmatrix}}^2 &= (D^2 - z^2)\sin^2(t) + (D^2 - z^2)\cos^2(t)\\

\text{If we have a density $\rho$, such that } \rho &= \rho(r),\\
\text{we can say } \rho(r) &= \rho(x,y,z) = \rho(\sqrt{x^2 + y^2 + z^2})\\
\text{The mass around the circle } &= \int_0^{2\pi} \rho(x(t),y(t),z(t))\sqrt{\dot{x}(t)^2 + \dot{y}(t)^2 + \dot{z}(t)^2} \; dt\\
\text{The mass around a sphere of radius D } &= \int_{-D}^{D} \frac{D}{\sqrt{D^2 - z^2}} \int_0^{2\pi} \rho(x(t),y(t),z)\sqrt{\dot{x}(t)^2 + \dot{y}(t)^2} \; dt \: dz \\
&= \int_{-D}^{D} \frac{D}{\sqrt{D^2 - z^2}} \int_0^{2\pi} \rho(\sqrt{(D^2 - z^2)\cos^2(t) + (D^2 - z^2)\sin^2(t) + z^2})\sqrt{\dot{x}(t)^2 + \dot{y}(t)^2} \; dt \: dz \\

&= \int_{-D}^{D} \frac{D}{\sqrt{D^2 - z^2}} \int_0^{2\pi} \rho(\sqrt{D^2 - z^2 + z^2})\sqrt{\dot{x}(t)^2 + \dot{y}(t)^2} \; dt \: dz \\

&= \int_{-D}^{D} \frac{D}{\sqrt{D^2 - z^2}} \int_0^{2\pi} \rho(D)\sqrt{\dot{x}(t)^2 + \dot{y}(t)^2} \; dt \: dz \\
\end{align*}
$$